In [ ]:
import json
from sympy import sympify

class Node:
    def __init__(self, degree):
        self.degree = degree
        self.coefficients = []
        self.next = None

class PolynomialList:
    def __init__(self):
        self.head = Node(None)
        self.head.next = self.head
        self.n = 0

    def get(self, degree):
        if degree >= self.size():
            raise Exception('Out of list')
        x = self.head.next
        while x.degree != degree:
            x = x.next
        return x

    def insert_after(self, x, coefficients):
        y = Node(len(coefficients) - 1)
        y.coefficients = coefficients
        self.n += 1
        y.next = x.next
        x.next = y
        return y

    def delete(self, x):
        if self.size() == 0:
            raise Exception('List is empty')
        self.n -= 1
        prev_node = self.head
        while prev_node.next != x:
            prev_node = prev_node.next
        prev_node.next = x.next
        return x

    def size(self):
        return self.n

    def is_empty(self):
        return self.n == 0

def get_coefficients(degree):
    while True:
        try:
            expression = input(f"Enter the polynomial for degree {degree} (e.g., 'x^2 + 3x + 2'): ")
            expression = expression.replace("^", "**")  # Convert '^' to '**' for exponentiation
            polynomial = sympify(expression)
            if polynomial.is_polynomial():
                coefficients = [polynomial.coeff(x) for x in reversed(range(degree + 1))]
                return coefficients
            else:
                raise ValueError
        except (ValueError, SyntaxError):
            print("Invalid input! Please enter a valid polynomial expression.")

def insert_polynomials(polynomial_list):
    print("Insert Polynomial Menu:")
    while True:
        parent_count = int(input("Enter the number of parent polynomials: "))
        for _ in range(parent_count):
            degree = int(input("Enter the degree of the polynomial: "))
            coefficients = get_coefficients(degree)
            polynomial_list.insert_after(polynomial_list.head, coefficients)

            child_count_expr = input("Enter the number of child polynomials for this parent (as an expression): ")
            child_count = int(eval(child_count_expr))
            for _ in range(child_count):
                coefficients = get_coefficients(degree)
                polynomial_list.insert_after(polynomial_list.head, coefficients)

            continue_choice = input("Do you want to continue adding polynomials? (yes/no): ")
            if continue_choice.lower() != 'yes':
                break

        main_menu_choice = input("Returning to Main Menu... Enter anything to continue: ")
        if main_menu_choice:
            break

    print("Returning to Main Menu...")

def multiply_polynomials(polynomial_list):
    print("Multiply Polynomials Menu:")
    print("Select the degree of parent polynomial to multiply:")
    degree = int(input("Enter the degree of the parent polynomial: "))

    # Find parent polynomials with the given degree
    parent_polynomials = [polynomial_list.get(i) for i in range(polynomial_list.size()) if polynomial_list.get(i).degree == degree]

    if not parent_polynomials:
        print("No parent polynomial found with the given degree.")
        return

    # Multiply all child polynomials of selected parent polynomials
    result = [1] * (2 * degree + 1)  # Initialize result polynomial with all coefficients as 1

    for parent_poly in parent_polynomials:
        current_node = parent_poly.next
        while current_node != parent_poly:
            for i in range(len(result)):
                if i < len(current_node.coefficients):
                    result[i] *= current_node.coefficients[i]
            current_node = current_node.next

    print("Result of multiplying selected polynomials:")
    print(result)

    input("Press Enter to continue...")


def sum_polynomials(polynomial_list):
    print("Sum Polynomials Menu:")
    print("Select the degree of parent polynomial to sum:")
    degree = int(input("Enter the degree of the parent polynomial: "))

    # Find parent polynomials with the given degree
    parent_polynomials = [polynomial_list.get(i) for i in range(polynomial_list.size()) if polynomial_list.get(i).degree == degree]

    if not parent_polynomials:
        print("No parent polynomial found with the given degree.")
        return

    # Sum all child polynomials of selected parent polynomials
    result = [0] * (degree + 1)  # Initialize result polynomial with all coefficients as 0

    for parent_poly in parent_polynomials:
        current_node = parent_poly.next
        while current_node != parent_poly:
            for i in range(len(result)):
                if i < len(current_node.coefficients):
                    result[i] += current_node.coefficients[i]
            current_node = current_node.next

    print("Result of summing selected polynomials:")
    print(result)

def display_all_polynomials(polynomial_list):
    print("Polynomials in the list:")
    current_node = polynomial_list.head.next
    while current_node != polynomial_list.head:
        print(f"Degree: {current_node.degree}, Coefficients: {current_node.coefficients}")
        current_node = current_node.next

def save_polynomials(polynomial_list, filename):
    with open(filename, 'w') as file:
        polynomial_data = []
        current_node = polynomial_list.head.next
        while current_node != polynomial_list.head:
            polynomial_data.append({
                'degree': current_node.degree,
                'coefficients': current_node.coefficients
            })
            current_node = current_node.next
        json.dump(polynomial_data, file)

def load_polynomials(filename):
    polynomial_list = PolynomialList()
    try:
        with open(filename, 'r') as file:
            polynomial_data = json.load(file)
            for data in polynomial_data:
                polynomial_list.insert_after(polynomial_list.head, data['coefficients'])
    except FileNotFoundError:
        pass
    return polynomial_list

def exit_program(polynomial_list):
    save_polynomials(polynomial_list, "polynomials.json")
    print("Exiting...")

def main_menu():
    print("Main Menu:")
    print("1. Insert Polynomials")
    print("2. Multiply Polynomials")
    print("3. Sum Polynomials")
    print("4. Display Polynomials")
    print("5. Exit")

def main():
    polynomial_list = load_polynomials("polynomials.json")
    menu_options = {
        '1': insert_polynomials,
        '2': multiply_polynomials,
        '3': sum_polynomials,
        '4': display_all_polynomials,
        '5': exit_program
    }

    while True:
        main_menu()
        choice = input("Enter your choice: ")
        if choice in menu_options:
            if choice == '5':
                menu_options[choice](polynomial_list)
                break
            else:
                menu_options[choice](polynomial_list)
        else:
            print("Invalid choice!")

if __name__ == "__main__":
    main()

Main Menu:
1. Insert Polynomials
2. Multiply Polynomials
3. Sum Polynomials
4. Display Polynomials
5. Exit
Enter your choice: 1
Insert Polynomial Menu:
Enter the number of parent polynomials: 1
Enter the degree of the polynomial: 2
Enter the polynomial for degree 2 (e.g., 'x^2 + 3x + 2'): ax^2+bx+c
Enter the number of child polynomials for this parent (as an expression): 2
Enter the polynomial for degree 2 (e.g., 'x^2 + 3x + 2'): yx^3+cx^2+tx+h
Enter the polynomial for degree 2 (e.g., 'x^2 + 3x + 2'): ux^3+tx^2+qx+p
Do you want to continue adding polynomials? (yes/no): no
Returning to Main Menu... Enter anything to continue: 1
Returning to Main Menu...
Main Menu:
1. Insert Polynomials
2. Multiply Polynomials
3. Sum Polynomials
4. Display Polynomials
5. Exit
Enter your choice: 4
Polynomials in the list:
Degree: 2, Coefficients: [0, p + qx + tx**2 + ux**3, 0]
Degree: 2, Coefficients: [0, cx**2 + h + tx + yx**3, 0]
Degree: 2, Coefficients: [0, ax**2 + bx + c, 0]
Main Menu:
1. Insert Polyn